# Library import

In [1]:
import os
import subprocess
import time

import yaml
from sklearn.model_selection import GroupKFold

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch.nn as nn

### Clona il repository

In [2]:
!git clone https://github.com/ultralytics/yolov5  

Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 17075 (delta 27), reused 26 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17075/17075), 15.69 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (11721/11721), done.


### Cambia directory

In [3]:
%cd /kaggle/working/yolov5

/kaggle/working/yolov5


### Installa le dipendenze

In [4]:
!pip install -r requirements.txt                  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 15.2 MB/s eta 0:00:00a 0:00:01


## Path

In [5]:
train_file = "/kaggle/input/our-xview-dataset/YOLO_cfg/train.txt"
val_file = "/kaggle/input/our-xview-dataset/YOLO_cfg/val.txt"
test_file = "/kaggle/input/our-xview-dataset/YOLO_cfg/test.txt"
dataset_yaml = "/kaggle/input/our-xview-dataset/YOLO_cfg/xview_yolo.yaml" # bisogna modificare i path nel file perchè non si trovano 
                                                                            # -> bisogna anche vedere se fare delle modifiche alle classi


# path per la gestione del modello e dell'addestramento
model_path = "yolov5s.pt"


# Network

In [6]:
class YoloModel(nn.Module):
    """
    Classe YOLOv5 per definire il modello e la funzione di forward.
    """
    def __init__(self, model_path="yolov5s.pt"):
        """
        Inizializza il modello YOLOv5.

        Args:
            model_path (str): Percorso ai pesi pre-addestrati YOLOv5.
        """
        super(YoloModel, self).__init__()
        self.model_path = model_path
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    def forward(self, images):
        """
        Esegue la predizione sul batch di immagini.

        Args:
            images (torch.Tensor): Batch di immagini di input.

        Returns:
            torch.Tensor: Risultati delle predizioni.
        """
        return self.model(images)


In [7]:
yolo_model = YoloModel() # non è necessario segnalare il numero di classi perchè sono prese direttamente dal file .yaml

/opt/conda/lib/python3.10/site-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-12-8 Python-3.10.14 torch-2.4.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 33.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## per addestrare il modello
### python train.py --img 640 --epochs 3 --data dataset.yaml --weights yolov5s.pt

In [8]:
class Trainer:
    """
    Classe per addestrare un modello YOLOv5.
    """
    def __init__(self, model, dataset_yaml, optimizer, img_size=640, batch_size=16, epochs=50, cache="ram", save_best_only=True):
        """
        Inizializza il Trainer per YOLOv5.

        Args:
            model (YoloModel): Istanza del modello YOLOv5.
            dataset_yaml (str): Percorso al file di configurazione del dataset.
            optimizer (str): Ottimizzatore (e.g., 'Adam', 'SGD').
            img_size (int): Dimensione delle immagini di input.
            batch_size (int): Dimensione del batch per l'addestramento.
            epochs (int): Numero di epoche.
            cache (str): Tipo di caching ('ram' o 'disk').
            save_best_only (bool): Se True, salva solo il modello con la migliore performance di validazione.
        """
        self.model = model
        self.dataset_yaml = dataset_yaml
        self.img_size = img_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.cache = cache
        self.optimizer = optimizer
        self.save_best_only = save_best_only
        self.best_val_loss = float('inf')  # Inizializza la miglior loss di validazione come infinita

    def train(self): 
        """
        Avvia l'addestramento del modello utilizzando YOLOv5.
        """
        print("Inizio addestramento...")
        for epoch in range(self.epochs):
            # Definisci il comando per l'addestramento
            command = (
                f"python train.py --img {self.img_size} --batch-size {self.batch_size} "
                f"--epochs {self.epochs} --optimizer {self.optimizer} "
                f"--data {self.dataset_yaml} "
                f"--weights {self.model.model_path} --cache {self.cache} "
                f"--save-period 1 --project runs/train --name {self.model.name} --exist-ok"
            )
            try:
                # Avvia il processo di training e monitora la validazione
                subprocess.run(command, check=True, shell=True)
                print(f"Epoch {epoch+1}/{self.epochs} completata.")
            except subprocess.CalledProcessError as e:
                print(f"Errore durante l'addestramento, Epoch {epoch+1}/{self.epochs} fallita. Dettagli: {e}")
                break

            # A questo punto, possiamo eseguire la validazione per monitorare i progressi
            val_loss = self.validate()

            # Salva il miglior modello sulla base della validazione
            if self.save_best_only and val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.save_best_model()

        print("Addestramento completato.")

    def validate(self):
        """
        Valida il modello sui dati di test e ritorna la loss di validazione.

        Returns:
            float: La loss di validazione.
        """
        print("Inizio validazione...")
        command = f"python val.py --data {self.dataset_yaml} --weights {self.model.model_path} --img {self.img_size}"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        # Estrai la loss di validazione dal log di output
        for line in result.stdout.splitlines():
            if "val_loss" in line:  # Trova la linea con la validazione della loss
                val_loss = float(line.split()[-1])  # Assumendo che l'ultima colonna contenga la loss
                print(f"Loss di validazione: {val_loss}")
                return val_loss
        
        print("Non è stato possibile estrarre la loss di validazione.")
        return float('inf')  # Se non riesci a trovare la loss, ritorna infinito

    def save_best_model(self):
        """
        Salva il miglior modello sulla base della validazione.
        """
        print(f"Salvataggio del miglior modello con perdita di validazione: {self.best_val_loss}")
        best_model_path = f"best_model_{self.best_val_loss:.4f}.pt"
        # Comando per salvare il modello
        command = f"cp {self.model.model_path} {best_model_path}"
        subprocess.run(command, shell=True, check=True)
        print(f"Miglior modello salvato in {best_model_path}.")


In [9]:
trainer = Trainer(
    model=yolo_model,
    dataset_yaml=dataset_yaml,
    optimizer = "Adam",
    img_size=640,
    batch_size=16,
    epochs=50,
    cache="ram"
)

# 4. Avvia il training
trainer.train()

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
train: weights=yolov5s.pt, cfg=, data=/kaggle/input/our-xview-dataset/YOLO_cfg/xview_yolo.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github

Addestramento completato.
